In [1]:
import math
import json
import datetime as time

In [2]:
currentSave = "1_9 Science"
fileName = "NoFile"
building = "vab"

def getName() :
    inputName = input()
    if len(inputName) > 0 :
        return inputName
    else :
        print("must enter name.")
        return getName()

fileName = getName()
print(fileName)

 Mun Scanner 1


Mun Scanner 1


In [8]:
jsonPath = "D:\Games\steamapps\common\Kerbal Space Program\Ships\Script\PartData\\KerbalElectricalData.json"
baseDir = "D:\Games\steamapps\common\Kerbal Space Program\saves"
VABPath = "Ships\VAB"
SPHPath = "Ships\SPH"
directory = 'D:\Games\steamapps\common\Kerbal Space Program\saves\Ship Testing\Ships\VAB\\'
fileExt = ".craft"

fullPath = "D:\Games\steamapps\common\Kerbal Space Program\saves\Ship Testing\Ships\VAB\ElectTest1.craft"

def buildPath(baseDir, curSave, building, file, ext) :
    if building == "vab" :
        return baseDir + "\\" + curSave + "\\" + VABPath + "\\" + file + ext
    elif building == "sph" :
        return baseDir + "\\" + curSave + "\\" + SPHPath + "\\" + file + ext


fullPath = buildPath(baseDir, currentSave, building, fileName, fileExt)
print(fullPath)

D:\Games\steamapps\common\Kerbal Space Program\saves\1_9 Science\Ships\VAB\Mun Scanner 1.craft


In [13]:
parts = []

def parseData(rawData) :
    data = []
    lines = rawData.split("\n")
    for d in lines :
        splitData = d.strip("\t").split(" = ")
        for spl in splitData :
            spl.strip()
        if len(splitData) > 1 :
            data.append(splitData)
    return data

def getPartNames(data) :
    output = []
    for d in data :
        if d[0] == "part" :
            d[1] = d[1].split("_")[0]
            output.append(d[1])
    return output

with open(fullPath) as file :
    data = []
    rawData = file.read()
    data = parseData(rawData)
    parts = getPartNames(data)


# for p in partNames :
#     parts.append(p[1])
    
### Dealing with JSON Library and loading the needed data ###

def parseInt(x) :
    try :
        return int(x)
    except :
        return 11111111
    
def parseFloat(x) :
    try :
        return float(x)
    except :
        return 111111111111
    
class EData :
    name = ""
    FileName = ""
    Load = 0
    LoadScale = "sec"
    RWLoad = 0
    RwScale = "sec"
    Production = 0
    Battery = 0
    
    def __init__(self, name="", FileName="", Load=0, LoadScale="sec", RWLoad=0, RWScale="sec", Production=0, Battery=0) :
        self.name = name
        self.FileName = FileName
        self.Load = Load
        self.LoadScale = LoadScale
        self.RWLoad = RWLoad
        self.RWScale = RWScale
        self.Production = Production
        self.Battery = Battery
        
    def parseDict(d) :
        return EData(d["name"],d["FileName"],d["Load"],d["LoadScale"],d["RWLoad"],d["RWScale"],d["Production"],d["Battery"])
    
    def ToString(self) :
        return f"{self.name} \n   File Name: {self.FileName} \n   Load: {self.Load} ec/{self.LoadScale} \n   RWLoad: {self.RWLoad} ec/{self.RWScale} \n   Prod: {self.Production} ec/sec \n   Batt: {self.Battery} ec\n"
    
with open(jsonPath) as jsonFile :
    output = json.load(jsonFile, parse_int=parseInt, parse_float=parseFloat)
    
allEData = []
for i in output :
    for j in output[i] :
        allEData.append(EData.parseDict(j))
        
### Comparing the craft data to the json data and getting a true parts list ###

def getCalcPartsList(partsList) :
    output = []
    for p in partsList :
        for e in allEData :
            if e.FileName == p :
                output.append(e)
    return output

calcPartsList = []

calcPartsList = getCalcPartsList(parts)

### Calculator ###

class Calculator :
    parts = []
    
    totalLoad = 0
    totalProd = 0
    totalBatt = 0
    
    chargeRate = 0
    chargeTime_sec = 0
    chargeTime_time = None
    
    def __init__(self, parts) :
        self.clear()
        self.parts = parts
        
    def calculate(self) :
        self.sumTotals()
        self.calcChargeRate()
        self.calcChargeTime()
        self.convertTime()
    
    def sumTotals(self) :
        for p in self.parts :
            self.totalLoad += self.convertSeconds(p.Load, p.LoadScale)
            self.totalLoad += self.convertSeconds(p.RWLoad, p.RWScale)
            self.totalProd += p.Production
            self.totalBatt += p.Battery
            
    def calcChargeRate(self) :
        self.chargeRate = self.totalProd - self.totalLoad
    
    def calcChargeTime(self) :
        if self.chargeRate != 0 :
            self.chargeTime_sec = self.totalBatt / self.chargeRate
        else :
            self.chargeTime_sec = 0
    
    def convertSeconds(self, x, per) :
        if per == "min" :
            return x / 60
        elif per == "hr" :
            return x / math.pow(60, 2)
        else :
            return x
        
    def convertTime(self) :
        self.chargeTime_time = time.timedelta(seconds=self.chargeTime_sec)
        
    def ToString(self) :
        partsStr = ""
        for p in self.parts :
            partsStr += f"{p.ToString()}\n"
        return f"""Load : {round(self.totalLoad, 6)} ec/sec
Prod : {round(self.totalProd, 6)} ec/sec
Batt : {round(self.totalBatt, 6)} ec
        
Charge Rate : {round(self.chargeRate, 6)} ec/sec
Charge Secs : {round(self.chargeTime_sec, 6)} sec
Charge Time : {self.chargeTime_time}

Parts :
{partsStr}"""
    
    def clear(self) :
        self.parts.clear()
        self.totalLoad = 0
        self.totalProd = 0
        self.totalBatt = 0
        self.chargeRate = 0
        self.chargeTime_sec = 0
        self.chargeTime_time = None

### Calculate Output ###

calculator = Calculator(calcPartsList)

calculator.calculate()
print(calculator.ToString())

Load : 1.4225 ec/sec
Prod : 2.8 ec/sec
Batt : 610 ec
        
Charge Rate : 1.3775 ec/sec
Charge Secs : 442.831216 sec
Charge Time : 0:07:22.831216

Parts :
Probodobodyne_OKTO 
   File Name: probeCoreOcto.v2 
   Load: 1.2 ec/min 
   RWLoad: 1.8 ec/min 
   Prod: 0 ec/sec 
   Batt: 10 ec

OX-STAT_Photovoltaic_Panels 
   File Name: solarPanels5 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0.35 ec/sec 
   Batt: 0 ec

OX-STAT_Photovoltaic_Panels 
   File Name: solarPanels5 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0.35 ec/sec 
   Batt: 0 ec

OX-STAT_Photovoltaic_Panels 
   File Name: solarPanels5 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0.35 ec/sec 
   Batt: 0 ec

OX-STAT_Photovoltaic_Panels 
   File Name: solarPanels5 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0.35 ec/sec 
   Batt: 0 ec

Small_RW 
   File Name: sasModule 
   Load: 0 ec/sec 
   RWLoad: 0.25 ec/sec 
   Prod: 0 ec/sec 
   Batt: 0 ec

Z-200 
   File Name: batteryBankMini 
   Load: 0 ec/sec 
   RWLo